In [1]:
from xno.data.datasets.hdf5_dataset import H5pyDataset
from utils import MatReader
from torch.utils.data import DataLoader, TensorDataset, Dataset, default_collate
import numpy as np
from timeit import default_timer

In [3]:
import torch
import matplotlib.pyplot as plt
import sys
from xno.models import XNO
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalXNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss

In [4]:
from xno.data.transforms.normalizers import UnitGaussianNormalizer

In [5]:
data_path = 'data/NavierStokes_V1e-5_N1200_T20.mat'

In [6]:
ntrain = 1000
ntest = 200

modes = 8
width = 20

batch_size = 10
batch_size2 = batch_size

epochs = 500
learning_rate = 0.001
scheduler_step = 100
scheduler_gamma = 0.5

print(epochs, learning_rate, scheduler_step, scheduler_gamma)

runtime = np.zeros(2, )
t1 = default_timer()

sub = 1
S = 64 // sub
T_in = 10
T = 10


500 0.001 100 0.5


In [7]:
train_u.shape[-1], T, train_u.shape

NameError: name 'train_u' is not defined

In [8]:
reader = MatReader(data_path)
train_a = reader.read_field('u')[:ntrain,::sub,::sub,:T_in]
train_u = reader.read_field('u')[:ntrain,::sub,::sub,T_in:T+T_in]

reader = MatReader(data_path)
test_a = reader.read_field('u')[-ntest:,::sub,::sub,:T_in]
test_u = reader.read_field('u')[-ntest:,::sub,::sub,T_in:T+T_in]

print(train_u.shape)
print(test_u.shape)
assert (S == train_u.shape[-2])
assert (T == train_u.shape[-1])

a_normalizer = UnitGaussianNormalizer(dim=[0, 1, 2, 3])
a_normalizer.fit(train_a)
train_a = a_normalizer.transform(train_a)
test_a = a_normalizer.transform(test_a)

y_normalizer = UnitGaussianNormalizer(dim=[0, 1, 2, 3]) 
y_normalizer.fit(train_u)  
train_u = y_normalizer.transform(train_u) 

train_a = train_a.reshape(ntrain,S,S,1,T_in).repeat([1,1,1,T,1])
test_a = test_a.reshape(ntest,S,S,1,T_in).repeat([1,1,1,T,1])


torch.Size([1000, 64, 64, 10])
torch.Size([200, 64, 64, 10])


In [9]:
# Define the custom Dataset
class DictDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {'x': self.x[idx], 'y': self.y[idx]}

In [10]:
train_a.shape, train_u.shape, test_a.shape, test_u.shape

(torch.Size([1000, 64, 64, 10, 10]),
 torch.Size([1000, 64, 64, 10]),
 torch.Size([200, 64, 64, 10, 10]),
 torch.Size([200, 64, 64, 10]))

In [11]:
train_a = train_a.permute(0, 4, 3, 2, 1)
train_u = train_u.unsqueeze(1)
train_u = train_u.permute(0, 1, 4, 3, 2)
test_a = test_a.permute(0, 4, 3, 2, 1)
test_u = test_u.unsqueeze(1)
test_u = test_u.permute(0, 1, 4, 3, 2)

In [12]:
train_a.shape, train_u.shape, test_a.shape, test_u.shape

(torch.Size([1000, 10, 10, 64, 64]),
 torch.Size([1000, 1, 10, 64, 64]),
 torch.Size([200, 10, 10, 64, 64]),
 torch.Size([200, 1, 10, 64, 64]))

In [40]:
train_loader = DictDataset(train_a, train_u)
test_loader = DictDataset(test_a, test_u)

In [41]:
train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)
test_loader = {
    64: test_loader
}

In [42]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

(torch.utils.data.dataloader.DataLoader,
 dict,
 torch.Size([10, 10, 10, 64, 64]),
 torch.Size([10, 1, 10, 64, 64]))

In [44]:
batch = next(iter(test_loader[64]))
type(test_loader), type(batch), batch['x'].shape, batch['y'].shape

(dict, dict, torch.Size([10, 10, 10, 64, 64]), torch.Size([10, 1, 10, 64, 64]))

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
model = XNO(
    max_n_modes=(16, 16, 16),
    n_modes=(modes, modes, modes),
    hidden_channels=width,
    in_channels=10,
    out_channels=1,
    transformation="fno",
    # transformation_kwargs={"wavelet_level": level, "wavelet_size": [40], "wavelet_filter":[wavelet]}, 
)
model = model.to(device)
n_params = count_model_params(model)

======== Selected Kernel Description =======
Dimentionality: 3D
Transformation: [ Fourier Neural Operator (FNO) Kernel ]
>>> Overview:
The FNO leverages Fourier Transform to map input data into the spectral domain, where
convolutional operations are performed by truncating high-frequency modes.

>>> Key Features:
- Effective for parameterized Partial Differential Equations (PDEs).
- Reduces computational complexity by retaining only significant modes.

>>> Reference:
Li, Z. et al. 'Fourier Neural Operator for Parametric Partial Differential Equations' (ICLR 2021).
Link: https://arxiv.org/pdf/2010.08895

================== Config ==================
>>> Normaliztion: None
>>> Activation Function: 



In [47]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [48]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=2048,
    dataset_indices=[2],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

Original Incre Res: change index to 0
Original Incre Res: change sub to 2
Original Incre Res: change res to 1024


In [49]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()


### N PARAMS ###
 13112941

### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###

 * Train: <xno.losses.data_losses.H1Loss object at 0x173c988b0>

 * Test: {'h1': <xno.losses.data_losses.H1Loss object at 0x173c988b0>, 'l2': <xno.losses.data_losses.LpLoss object at 0x3158d26e0>}


In [50]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [ ]:
trainer.train(
    train_loader,
    test_loader,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
    save_every=5, 
    save_dir="save/lorenz1d_lno"
)

Training on 1000 samples
Testing on [200] samples         on resolutions [64].
Raw outputs of shape torch.Size([10, 1, 5, 64, 64])


In [21]:
# FNO


In [22]:
# HNO


In [ ]:
# WNO


In [24]:
# LNO
